In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
import sys
from glob import glob
from pprint import pprint

sys.path.append("/home/phornauer/Git/axon_tracking/")
from axon_tracking import template_extraction as te

In [5]:
te_params = dict()
te_params['n_jobs'] = 16 #Number of cores to use for waveform extraction
te_params['filter_band'] = 150 #Either float for the highpass filter frequency or list for the bandpass filter frequencies
te_params['overwrite'] = False #Flag if templates should be recalculated if already existing
te_params['max_spikes_per_unit'] = 1000 #Maximum number of spikes to be used for template extraction

qc_params = dict()
qc_params['min_n_spikes'] = 1500 #Minimum number of spikes to be detected for a unit for template extraction to take place
qc_params['exclude_mua'] = True #Exclude units that were labelled multi unit activity by kilosort
qc_params['use_bc'] = True #Use bombcell for QC
qc_params['use_si'] = True #Use spikeinterface for QC
qc_params["auto_merge"] = True #Automatically merge units (spikeinterface implementation)
qc_params["remove_redundant"] = True #Remove redundant units (spikeinterface implementation)

We only need a list of folders containing the sorting results (sorting_list) and can then infer the recording paths automatically. Change root_path and path_pattern so that they represent the sortings you want to analyze. 
Use wildcards (*) where appropriate, but still try to be as specific as possible to prevent ambiguities.

In [6]:
root_path = "/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/" # Fixed path root that all recordings have in common
path_pattern = ["EI_iNeurons", "24*",  "*","AxonTracking","w*","sorter_output"] # Variable part of the path, where we collect all possible combinations using wildcards (*). It is still recommended to be as specific as possible to avoid ambiguities.

full_path = os.path.join(root_path, *path_pattern)
sorting_list = glob(full_path)
sorting_list.sort()
print(f'Found {len(sorting_list)} sorting paths matching the description:\n{full_path}\n')
#pprint(sorting_list)

Found 60 sorting paths matching the description:
/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/EI_iNeurons/24*/*/AxonTracking/w*/sorter_output



Optional: Check the sorting list and only include a selection of the sortings

In [ ]:
pprint(sorting_list)

In [ ]:
sorting_list = sorting_list[:1] # For testing purposes, we only use the first recording

Now we can run the actual template extraction:

In [7]:
te.extract_templates_from_sorting_list(sorting_list, qc_params, te_params)

  0%|          | 0/60 [00:00<?, ?it/s]

Keeping 18 good units


  2%|▏         | 1/60 [16:02<15:46:15, 962.30s/it]

Keeping 19 good units


  3%|▎         | 2/60 [31:24<15:07:41, 938.99s/it]

...

 97%|█████████▋| 58/60 [9:16:37<18:54, 567.38s/it]

Keeping 15 good units


 98%|█████████▊| 59/60 [9:26:00<09:26, 566.16s/it]

Keeping 7 good units


100%|██████████| 60/60 [9:35:31<00:00, 575.52s/it]


We can also have a quick look at the template we just extracted:

In [ ]:
from axon_tracking import visualization as vis #Separate as it is not needed for the template extraction

In [ ]:
root_path = "/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/Torsten_2/241010/T002523/AxonTracking/well000/sorter_output/"
plot_path = os.path.join(root_path, 'templates')

vis.plot_template_overview(plot_path)